In [1]:
from pre_proc import *
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
names =['symboling', 'normalized-losses', 'make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location','wheel-base','length','width','height','curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']

In [3]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', names=names)

### Data exploration

In [4]:
df.iloc[40:55]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
40,0,85,honda,gas,std,four,sedan,fwd,front,96.5,...,110,1bbl,3.15,3.58,9.0,86,5800,27,33,10295
41,0,85,honda,gas,std,four,sedan,fwd,front,96.5,...,110,mpfi,3.15,3.58,9.0,101,5800,24,28,12945
42,1,107,honda,gas,std,two,sedan,fwd,front,96.5,...,110,2bbl,3.15,3.58,9.1,100,5500,25,31,10345
43,0,?,isuzu,gas,std,four,sedan,rwd,front,94.3,...,111,2bbl,3.31,3.23,8.5,78,4800,24,29,6785
44,1,?,isuzu,gas,std,two,sedan,fwd,front,94.5,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
45,0,?,isuzu,gas,std,four,sedan,fwd,front,94.5,...,90,2bbl,3.03,3.11,9.6,70,5400,38,43,?
46,2,?,isuzu,gas,std,two,hatchback,rwd,front,96.0,...,119,spfi,3.43,3.23,9.2,90,5000,24,29,11048
47,0,145,jaguar,gas,std,four,sedan,rwd,front,113.0,...,258,mpfi,3.63,4.17,8.1,176,4750,15,19,32250
48,0,?,jaguar,gas,std,four,sedan,rwd,front,113.0,...,258,mpfi,3.63,4.17,8.1,176,4750,15,19,35550
49,0,?,jaguar,gas,std,two,sedan,rwd,front,102.0,...,326,mpfi,3.54,2.76,11.5,262,5000,13,17,36000


In [5]:
df = df.replace('?', np.nan)

In [6]:
df['normalized-losses'] = pd.to_numeric(df['normalized-losses'])
df['bore'] = pd.to_numeric(df['bore'])
df['stroke'] = pd.to_numeric(df['stroke'])
df['horsepower'] = pd.to_numeric(df['horsepower'])
df['peak-rpm'] = pd.to_numeric(df['peak-rpm'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    164 non-null float64
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         203 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression-ratio    205 non-null float64
horsepower           203 non-

In [8]:
df.describe()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
count,205.000000,164.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,201.000000,201.000000,205.000000,203.000000,203.000000,205.000000,205.000000
mean,0.834146,122.000000,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329751,3.255423,10.142537,104.256158,5125.369458,25.219512,30.751220
std,1.245307,35.442168,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.273539,0.316717,3.972040,39.714369,479.334560,6.542142,6.886443
min,-2.000000,65.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000
25%,0.000000,94.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000
50%,1.000000,115.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000
75%,2.000000,150.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.590000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000
max,3.000000,256.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000


In [9]:
df.isna().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

### deal with NaNs

In [170]:
### Drop all NaNs in table
fr = drop_nan(df)

In [171]:
### Select target (price)
targets = pd.to_numeric(fr['price'])
fr = fr.drop(labels=['price'], axis = 1)

### Select ony numeric features
numeric = ['symboling','normalized-losses','wheel-base','length','width','height','curb-weight','engine-size','bore','stroke',
           'compression-ratio', 'horsepower','peak-rpm','city-mpg','highway-mpg']
dataset_num = fr[numeric]

### Normalize all data
dataset_num = normalize(dataset_num)

In [172]:
### Split sets
X_train, X_test, y_train, y_test = train_test_split(dataset_num, targets, test_size=0.2,random_state=4)

### Train model
reg = linear_model.Lasso(alpha=0.1)
reg = reg.fit(X_train,y_train)

### Test prediction
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.689964974660933

### Better preprocecing

In [173]:
### Drop all NaNs in target feature
fr = drop_nan(df,colum='price')

### replace NaNs in bore, stroke, horsepower
fr = repl_nan(fr,colum='bore',type='median')
fr = repl_nan(fr,colum='stroke',type='mode')
fr = repl_nan(fr,colum='horsepower',type='mean')

### replace NaNs by linear reggresion in normalized-losses base on price
fr = rep_nan_reg(fr.reset_index(),to_replace='normalized-losses',base_on='price')

### replace NaNs by KNN in peak-rpm base on wheel-base,engine-size,normalized-losses
fr = KNN(fr,['wheel-base','engine-size','normalized-losses'],'peak-rpm',k=7,count_neib=10)

C:\Users\Mudryi-LP\pre_proc.py:93: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  x = x_train.as_matrix().reshape(-1,1)
C:\Users\Mudryi-LP\pre_proc.py:94: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  y = y_train.as_matrix().reshape(-1,1)
C:\Users\Mudryi-LP\pre_proc.py:97: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  y_test = y_test.as_matrix().reshape(-1,1)
C:\Users\Mudryi-LP\pre_proc.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[to_replace][i] = a[k][0]


In [174]:
### Select target (price)
targets = pd.to_numeric(fr['price'])
fr = fr.drop(labels=['price'], axis = 1)

### Select ony numeric features
numeric = ['symboling','normalized-losses','wheel-base','length','width','height','curb-weight','engine-size','bore','stroke',
           'compression-ratio', 'horsepower','city-mpg','highway-mpg']
dataset_num = fr[numeric]

### Normalize and standardize some data 
dataset_num = normalize(dataset_num,colum='symboling')
dataset_num = normalize(dataset_num,colum='length')
dataset_num = normalize(dataset_num,colum='wheel-base')
dataset_num = normalize(dataset_num,colum='city-mpg')
dataset_num = normalize(dataset_num,colum='highway-mpg')
dataset_num = normalize(dataset_num,colum='bore')
dataset_num = normalize(dataset_num,colum='stroke')
dataset_num = normalize(dataset_num,colum='width')
dataset_num = normalize(dataset_num,colum='height')
dataset_num = standardize(dataset_num,colum='horsepower')
dataset_num = standardize(dataset_num,colum='engine-size')
dataset_num = standardize(dataset_num,colum='normalized-losses')
dataset_num = standardize(dataset_num,colum='compression-ratio')
dataset_num = standardize(dataset_num,colum='curb-weight')

In [175]:
### Split sets
X_train, X_test, y_train, y_test = train_test_split(dataset_num, targets, test_size=0.2,random_state=4)

### Train model
reg = linear_model.Lasso(alpha=0.1)
reg = reg.fit(X_train,y_train)

### Test prediction
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.816764203982569